In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/fb_caltech_small_attrlist.csv')
df.head()

,year0,year1968,year1976,year1979,year1984,year1993,year1996,year1999,year2001,year2002,...,major217,major220,major221,major222,major223,major224,major226,major227,major228,major229
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Compute the similarity matrix

In [2]:
def similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_vertex_attri(G, v):
    return list(G.vs[v].attributes().values())

def get_sim_between(G, i, j):
    return round(similarity(get_vertex_attri(G, i), get_vertex_attri(G, j)), 2)

In [3]:
def find_community(vertex, C):
    for c in C:
        if vertex in c:
            return c
    return []

def compute_mod_gain(vertex, community, sim_matrix, alpha):
    # Number of edges
    E = len(G.es)
    
    community = list(set(community))
    G_ij = 0  
    Q_attr = 0
    
    for member in community:
        Q_attr += sim_matrix[vertex][member]
        if G.are_connected(vertex, member):
            edge = G.get_eid(vertex, member)
            G_ij += G.es['weight'][edge]
            
    # print(G_ij)
    Q_newman = (G_ij - ((sum(G.degree(community)) * G.degree(vertex)) / (2 * E))) / (2 * E)
    
    # Normalize
    Q_attr = Q_attr / len(community)
    
    # print(Q_attr)
    # print (Q_newman, Q_attr)
    
    return alpha * Q_newman + (1 - alpha) * Q_attr
    
    

def phase_one(G, sim_matrix, C, alpha):
    changed = False
    V = G.vcount()
    
    for v in range(V):
        # Community of Vertex v
        C_i = find_community(v, C)
        
        # Max Modularity Gain
        max_mod_gain = -1
        C_new = []
        
        # c -> community
        # C -> Set of all communities
        for c in C:
            # Compute for a different cluster
            if set(c) != set(C_i):
                # Compute the modularity gain for community c when vertex v is added
                gain = compute_mod_gain(v, c, sim_matrix, alpha)
                if gain > 0 and gain > max_mod_gain:
                    max_mod_gain = gain
                    C_new = c
                
        
        if max_mod_gain > 0 and set(C_i) != set(C_new):
            # print(v, C_i, C_new)
            # Remove from older community
            C_i.remove(v)
            # Add to new community
            C_new.append(v)
            # Mark the change (count the number of changes)
            changed = True
            # if a community becomes empty, remove it from C
            if len(C_i) == 0:
                C.remove([])
                
    return changed

In [4]:
def run_phase_one(G, alpha, epochs):
    i = 0
    change = True
    # Vertices
    V = G.vcount()
    # Initial Communities
    C = [[x] for x in range(V)]
    # Similarity Matrix    
    sim_matrix = np.array([[get_sim_between(G, i, j) for j in range(V)] for i in range(V)])
    
    while change and i < epochs:
        change = phase_one(G, sim_matrix, C, alpha)
        print("Number of Communities after iteration {} - {}".format(i, len(C)))
        i += 1
        
    return C

In [5]:
def run_phase_two(G, C, alpha, epochs):
    V = G.vcount()
    print(V)

    new_vertex_map = [0 for x in range(V)]
    
    for idx, community in enumerate(C):
        for vertex in community:
            new_vertex_map[vertex] = idx

    # https://igraph.org/python/doc/igraph.GraphBase-class.html#contract_vertices
    G.contract_vertices(new_vertex_map, combine_attrs = "mean")
    # https://igraph.org/python/doc/igraph.GraphBase-class.html#simplify
    G.simplify(combine_edges = sum)

    print(G.vcount())
    
    new_communities = run_phase_one(G, alpha, epochs)
    return new_communities    

In [7]:
import igraph

# Edges
file = open('./data/fb_caltech_small_edgelist.txt')
E = []
for line in file:
    split = line.split(" ")
    E.append(tuple(map(int, split)))
    
# Vertices
V = len(df)

# Graph
G = igraph.Graph()
G.add_vertices(V)
G.add_edges(E)
G.es['weight'] = np.zeros((V, ), dtype = int)
for col in df.keys():
    G.vs[col] = df[col]

alpha = 0.5
epochs = 15
p1_communities = run_phase_one(G, alpha, epochs)
p2_communities = run_phase_two(G, p1_communities, alpha, epochs)
print(len(p2_communities))

Number of Communities after iteration 0 - 161
Number of Communities after iteration 1 - 122
Number of Communities after iteration 2 - 104
Number of Communities after iteration 3 - 87
Number of Communities after iteration 4 - 81
Number of Communities after iteration 5 - 77
Number of Communities after iteration 6 - 73
Number of Communities after iteration 7 - 69
Number of Communities after iteration 8 - 62
Number of Communities after iteration 9 - 57
Number of Communities after iteration 10 - 56
Number of Communities after iteration 11 - 54
Number of Communities after iteration 12 - 52
Number of Communities after iteration 13 - 50
Number of Communities after iteration 14 - 47
324
47
Number of Communities after iteration 0 - 25
Number of Communities after iteration 1 - 20
Number of Communities after iteration 2 - 17
Number of Communities after iteration 3 - 16
Number of Communities after iteration 4 - 14
Number of Communities after iteration 5 - 12
Number of Communities after iteration 6 

In [8]:
p2_communities

[[6, 16, 18, 22, 27, 43],
 [0, 1, 10, 19, 23, 34, 39, 41, 42, 45],
 [4, 7, 8, 12, 13, 14, 20, 28, 30, 32, 35, 36, 40],
 [2, 9, 11, 15, 21, 24, 26, 46],
 [25, 31, 38],
 [17, 37],
 [3, 33],
 [5, 29, 44]]

In [9]:
p1_communities

[[9, 22, 161, 173, 208, 236, 248],
 [1, 29, 88, 164, 287, 304],
 [24, 28, 44, 48, 76, 280],
 [21, 251, 301],
 [181, 277],
 [39, 56, 74, 86, 117, 128, 145, 281, 288, 290, 298],
 [26, 114, 130, 163, 179, 207, 215],
 [13, 17, 36, 57, 96, 120, 147, 158, 168, 204, 292],
 [53, 64, 68, 73, 151, 180, 205, 209, 268, 274, 310],
 [5, 23, 67, 104, 303],
 [25, 97, 253],
 [59, 127, 144, 165, 182, 189, 196, 313],
 [14, 31, 60, 82, 102, 111, 116, 170, 172, 262, 305, 318],
 [139],
 [8, 18, 35, 65, 75, 103, 188, 212],
 [63, 190, 213, 220, 233, 239, 260, 264, 289],
 [0, 7, 119, 136, 167, 193, 214, 218, 282],
 [54, 140, 228, 243],
 [12, 62, 78, 92, 99, 108, 132, 226, 286, 312],
 [30, 55, 58, 203, 238, 283, 294],
 [112, 148, 176, 223, 296, 308],
 [3, 115, 152, 232, 300],
 [71, 156, 175, 195, 216, 249, 258, 295, 299, 302],
 [80, 171, 183, 271, 275, 322],
 [95, 199, 210, 297, 316],
 [20, 38, 79, 90, 113, 124, 125, 126, 153, 231],
 [46, 121, 133, 150, 252, 266, 267, 306],
 [131, 160, 184, 224, 273, 319],
 [6,

In [12]:
mapped_communities = []
for community in p2_communities:
    temp = []
    for vertex in community:
        temp += p1_communities[vertex]
    mapped_communities.append(temp)
    
mapped_communities

[[26,
  114,
  130,
  163,
  179,
  207,
  215,
  0,
  7,
  119,
  136,
  167,
  193,
  214,
  218,
  282,
  12,
  62,
  78,
  92,
  99,
  108,
  132,
  226,
  286,
  312,
  71,
  156,
  175,
  195,
  216,
  249,
  258,
  295,
  299,
  302,
  131,
  160,
  184,
  224,
  273,
  319,
  43,
  185,
  187,
  192,
  198,
  230,
  261,
  276,
  307],
 [9,
  22,
  161,
  173,
  208,
  236,
  248,
  1,
  29,
  88,
  164,
  287,
  304,
  25,
  97,
  253,
  30,
  55,
  58,
  203,
  238,
  283,
  294,
  80,
  171,
  183,
  271,
  275,
  322,
  70,
  129,
  134,
  146,
  166,
  217,
  219,
  225,
  16,
  52,
  98,
  100,
  106,
  110,
  154,
  169,
  177,
  221,
  237,
  32,
  33,
  84,
  89,
  137,
  211,
  229,
  269,
  317,
  321,
  10,
  47,
  186,
  227,
  259,
  309,
  41],
 [181,
  277,
  13,
  17,
  36,
  57,
  96,
  120,
  147,
  158,
  168,
  204,
  292,
  53,
  64,
  68,
  73,
  151,
  180,
  205,
  209,
  268,
  274,
  310,
  14,
  31,
  60,
  82,
  102,
  111,
  116,
  170,
  172,
  26

In [13]:
file = open("communities.txt", 'w+')
for c in mapped_communities:
    for i in range(len(c)-1):
        file.write("%d," % c[i])
    file.write(str(c[-1]))
    file.write('\n')
file.close()